In [6]:
# Create the data directory if it doesn't already exist
!mkdir -p Data

# Download the file using wget and save it in the data directory
!wget https://github.com/maziarraissi/PINNs/raw/master/appendix/Data/burgers_shock.mat -O Data/burgers_shock.mat
!pip install pyDOE

--2024-03-28 20:47:04--  https://github.com/maziarraissi/PINNs/raw/master/appendix/Data/burgers_shock.mat
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/maziarraissi/PINNs/master/appendix/Data/burgers_shock.mat [following]
--2024-03-28 20:47:04--  https://raw.githubusercontent.com/maziarraissi/PINNs/master/appendix/Data/burgers_shock.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207944 (203K) [application/octet-stream]
Saving to: ‘Data/burgers_shock.mat’

Data/burgers_shock. 100%[===================>] 203.07K  --.-KB/s    in 0.02s   

2024-03-28 20:47:05 (7.94 MB/s) - ‘Data/burgers_shock.

In [7]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io
from scipy.spatial import Delaunay
from pyDOE import lhs
import time
from torch.autograd import Variable

In [3]:
class PhysicsInformedNN(nn.Module):
    def __init__(self, X_u, u, X_f, layers, lb, ub, nu):
        super(PhysicsInformedNN, self).__init__()

        # Convert numpy arrays to torch tensors
        self.lb = torch.Tensor(lb).float()
        self.ub = torch.Tensor(ub).float()

        self.x_u = torch.Tensor(X_u[:, 0:1]).float()
        self.t_u = torch.Tensor(X_u[:, 1:2]).float()
        self.x_f = torch.Tensor(X_f[:, 0:1]).float()
        self.t_f = torch.Tensor(X_f[:, 1:2]).float()
        self.u = torch.Tensor(u).float()

        self.layers = layers
        self.nu = nu

        # Neural Network
        self.model = self.neural_net(layers)
        self.optimizer = torch.optim.LBFGS(self.model.parameters(), lr=1.0,
                                           max_iter=50000, max_eval=50000,
                                           tolerance_grad=1.0*np.finfo(float).eps)

    def neural_net(self, layers):
        modules = []
        for i in range(len(layers) - 2):
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(layers[-2], layers[-1]))
        return nn.Sequential(*modules)

    def forward(self, x, t):
        u = self.model(torch.cat([x, t], dim=1))
        return u

    def net_f(self, x, t):
        u = self.forward(x, t)

        u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
        u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
        f = u_t + u*u_x - self.nu*u_xx

        return f

    def loss_fn(self):
        u_pred = self.forward(self.x_u, self.t_u)
        f_pred = self.net_f(self.x_f, self.t_f)

        loss = torch.mean((self.u - u_pred)**2) + torch.mean(f_pred**2)
        return loss

    def train(self):
        def closure():
            self.optimizer.zero_grad()
            loss = self.loss_fn()
            loss.backward()
            print('Loss:', loss.item())
            return loss

        self.optimizer.step(closure)

    def predict(self, X_star):
        self.model.eval()  # Set the model to evaluation mode
        X_star = torch.Tensor(X_star).float()
        u_star = self.forward(X_star[:, 0:1], X_star[:, 1:2])
        f_star = self.net_f(X_star[:, 0:1], X_star[:, 1:2])

        return u_star.detach().numpy(), f_star.detach().numpy()

In [9]:
# Set device for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

nu = 0.01/np.pi
noise = 0.0

N_u = 100
N_f = 10000
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

data = scipy.io.loadmat('./Data/burgers_shock.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

# Convert to torch tensors and move to device
X_u_train = torch.tensor(X_u_train, requires_grad=True).float().to(device)
u_train = torch.tensor(u_train).float().to(device)
X_f_train = torch.tensor(X_f_train, requires_grad=True).float().to(device)
lb = torch.tensor(lb).float().to(device)
ub = torch.tensor(ub).float().to(device)

model = PhysicsInformedNN(X_u_train, u_train, X_f_train, layers, lb, ub, nu).to(device)

start_time = time.time()
model.train()
elapsed = time.time() - start_time
print('Training time: %.4f' % elapsed)

# Predict
X_star = torch.tensor(X_star).float().to(device)
u_star = torch.tensor(u_star).float().to(device)

u_pred, f_pred = model.predict(X_star)

u_pred = u_pred.cpu().numpy() # Move predictions back to CPU for numpy operations
error_u = np.linalg.norm(u_star.cpu().numpy()-u_pred,2)/np.linalg.norm(u_star.cpu().numpy(),2)
print('Error u: %e' % (error_u))


Loss: 0.007883485406637192
Loss: 0.0077916886657476425
Loss: 0.007731166668236256
Loss: 0.007701439317315817
Loss: 0.007683803327381611
Loss: 0.007664700970053673
Loss: 0.007632147986441851
Loss: 0.007606790866702795
Loss: 0.007579836994409561
Loss: 0.0075400713831186295
Loss: 0.0074868518859148026
Loss: 0.007389322854578495
Loss: 0.007251276634633541
Loss: 0.00713195838034153
Loss: 0.0070107546634972095
Loss: 0.006902758963406086
Loss: 0.006838550791144371
Loss: 0.006783874239772558
Loss: 0.006745358929038048


KeyboardInterrupt: 